In [1]:
from TextMining_cls_wk_3 import *
from TextMining_ret_wk_3 import *
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification
from sklearn.model_selection import cross_val_score
from sklearn import linear_model, pipeline, feature_extraction

# Verify the results of Table 4.1

In [2]:
d = 0.1
pages = {'A':['B', 'C'], 'B':['A'], 'C':['A']}
rankings = {'A': 1, 'B': 1, 'C': 1}
def updateRanking():
    for k in pages:
        sumNeighbors = 0
        for neighbor in pages[k]:
            c = len(pages[neighbor])
            pr = rankings[neighbor]
            sumNeighbors += pr/c
        newRanking = d + (1 - d) * sumNeighbors
        rankings[k] = newRanking
print('Iteration 0', ['%.2f' % elem for elem in list(rankings.values())])
for i in range(1,26):
    updateRanking()
    print('Iteration', i, ['%.2f' % elem for elem in list(rankings.values())])

Iteration 0 ['1.00', '1.00', '1.00']
Iteration 1 ['1.90', '0.96', '0.96']
Iteration 2 ['1.82', '0.92', '0.92']
Iteration 3 ['1.75', '0.89', '0.89']
Iteration 4 ['1.70', '0.87', '0.87']
Iteration 5 ['1.66', '0.85', '0.85']
Iteration 6 ['1.62', '0.83', '0.83']
Iteration 7 ['1.59', '0.82', '0.82']
Iteration 8 ['1.57', '0.81', '0.81']
Iteration 9 ['1.55', '0.80', '0.80']
Iteration 10 ['1.54', '0.79', '0.79']
Iteration 11 ['1.53', '0.79', '0.79']
Iteration 12 ['1.52', '0.78', '0.78']
Iteration 13 ['1.51', '0.78', '0.78']
Iteration 14 ['1.50', '0.78', '0.78']
Iteration 15 ['1.50', '0.77', '0.77']
Iteration 16 ['1.49', '0.77', '0.77']
Iteration 17 ['1.49', '0.77', '0.77']
Iteration 18 ['1.49', '0.77', '0.77']
Iteration 19 ['1.48', '0.77', '0.77']
Iteration 20 ['1.48', '0.77', '0.77']
Iteration 21 ['1.48', '0.77', '0.77']
Iteration 22 ['1.48', '0.77', '0.77']
Iteration 23 ['1.48', '0.77', '0.77']
Iteration 24 ['1.48', '0.76', '0.76']
Iteration 25 ['1.48', '0.76', '0.76']


# retrieve the 10 best matches from the Reuters training XML. ‘create_document_vectors’ , ‘create_query_vectors’ ‘find_similar_k_docs’ 

In [3]:
traindata, vocab = create_document_vectors('tmsk_trn.properties', 300)

TMPY> Creating document vectors
TMPY> Tokenizing documents
TMPY> Removing stopwords
TMPY> Stemming tokens
TMPY> Generating dictionary
TMPY> Vectorizing documents


In [4]:
querydata = create_query_vectors(vocab,'tmsk_news.properties')

TMPY> Creating query vector
TMPY> Tokenizing documents
TMPY> Removing stopwords
TMPY> Stemming tokens
TMPY> Vectorizing documents


In [5]:
querydata.shape

(1, 452)

In [6]:
traindata.shape

(9603, 452)

In [7]:
kdocs = find_similar_k_docs(querydata, traindata, 10)
print(kdocs)

TMPY> Compute query-doc similarity
[5120, 3454, 8468, 6353, 9142, 6885, 8367, 9093, 3538, 129]


In [8]:
#print out the 10 best matches, they are all related to my query news
def replace_text(text, wordDic):
    rc = re.compile('|'.join(map(re.escape, wordDic)))
    def translate(match):
        return wordDic[match.group(0)]
    return rc.sub(translate, text)
def tokenize():      
    properties    = {}    
    f = open('tmsk_trn.properties', 'r')
    for line in f:
        cleanedLine = line.strip()
        if cleanedLine:
            if cleanedLine.startswith('#'):
                continue
            if len(cleanedLine.split('=')) == 2:
                params = cleanedLine.split('=')
                properties[params[0]]=params[1]
    f.close()  
    in_file    = open(properties['infile'],'r')
    lines      = in_file.readlines()
    in_file.close()
    sentences_raw = " ".join(lines)
    proc_dic = {'<D>':' ', '</D>':' '}    
    sentences = replace_text(sentences_raw,proc_dic).split("<")
    return sentences
def getdoc(sentences, id):
    doc_ids = [i for i, x in enumerate(sentences) if x.startswith('REUTERS') ]
    num_docs = len(doc_ids)
    if id == num_docs-1:
        current_start = doc_ids[id]
        current_end   = len(sentences)
    else:
        current_start = doc_ids[id]
        current_end   = doc_ids[id+1] - 1
    current_doc   = sentences[current_start:current_end:1]  
    print(current_doc)
sentences = tokenize()
for i in kdocs:
    getdoc(sentences, i)

['REUTERS TOPICS="YES" LEWISSPLIT="TRAIN" CGISPLIT="TRAINING-SET" OLDID="12239" NEWID="7326">\n ', 'DATE>19-MAR-1987 14:13:47.02', '/DATE>\n ', 'TOPICS> grain  wheat ', '/TOPICS>\n ', 'PLACES> usa  ussr ', '/PLACES>\n ', 'PEOPLE>', '/PEOPLE>\n ', 'ORGS>', '/ORGS>\n ', 'EXCHANGES>', '/EXCHANGES>\n ', 'COMPANIES>', '/COMPANIES>\n ', 'UNKNOWN> \n &#5;&#5;&#5;C G\n &#22;&#22;&#1;f2020&#31;reute\n b f BC-/U.S.-GRAIN-TRADE-CAL   03-19 0140', '/UNKNOWN>\n ', 'TEXT>&#2;\n ', 'TITLE>U.S. GRAIN TRADE CALLS SHULTZ REMARK SIGNIFICANT', '/TITLE>\n ', 'AUTHOR>    by Maggie McNeil, Reuters', '/AUTHOR>\n ', 'DATELINE>    WASHINGTON, March 19 - ', '/DATELINE>', 'BODY>A statement yesterday by Secretary\n of State George Shultz after he met with wheat growers that\n U.S. agricultural products must be competitively priced was\n significant in that he recognized the importance of the Soviet\n market and the need for U.S. prices to be at world market\n levels, U.S. grain trade industry officials said.\n    

['REUTERS TOPICS="YES" LEWISSPLIT="TRAIN" CGISPLIT="TRAINING-SET" OLDID="15303" NEWID="9782">\n ', 'DATE>26-MAR-1987 07:21:27.37', '/DATE>\n ', 'TOPICS> grain  wheat ', '/TOPICS>\n ', 'PLACES> uk  canada  usa  ussr ', '/PLACES>\n ', 'PEOPLE>', '/PEOPLE>\n ', 'ORGS> iwc-wheat  ec ', '/ORGS>\n ', 'EXCHANGES>', '/EXCHANGES>\n ', 'COMPANIES>', '/COMPANIES>\n ', 'UNKNOWN> \n &#5;&#5;&#5;C G\n &#22;&#22;&#1;f0556&#31;reute\n b f BC-IWC-LIFTS-WORLD-GRAIN   03-26 0118', '/UNKNOWN>\n ', 'TEXT>&#2;\n ', 'TITLE>IWC LIFTS WORLD GRAIN OUTPUT ESTIMATE TO RECORD', '/TITLE>\n ', 'DATELINE>    LONDON, March 26 - ', '/DATELINE>', 'BODY>The International Wheat Council (IWC)\n lifted its estimate for 1986/87 world wheat and coarse grain\n production by one mln tonnes to a record 1,377 mln, compared\n with 1,351 mln tonnes the previous season.\n     In its monthly market report, the IWC said it is leaving\n unchanged its forecast of world wheat production for the coming\n 1987/88 season at between 520 and 

# Use ‘cluster_k_means’ 10 clusters 20 clusters, then 30 clusters. optimal k? printing the silhouette coefficient score.

In [11]:
km10, score10 = cluster_k_means(traindata,10)
km20, score20 = cluster_k_means(traindata,20)
km30, score30 = cluster_k_means(traindata,30)

TMPY> Clustering document vectors
Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 866160.9321297258
start iteration
done sorting
end inner loop
Iteration 1, inertia 837706.2162538565
start iteration
done sorting
end inner loop
Iteration 2, inertia 831915.6237019908
start iteration
done sorting
end inner loop
Iteration 3, inertia 829531.292724687
start iteration
done sorting
end inner loop
Iteration 4, inertia 827839.6918679936
start iteration
done sorting
end inner loop
Iteration 5, inertia 826702.7286135036
start iteration
done sorting
end inner loop
Iteration 6, inertia 825922.905996963
start iteration
done sorting
end inner loop
Iteration 7, inertia 825411.0895113511
start iteration
done sorting
end inner loop
Iteration 8, inertia 825131.0806345171
start iteration
done sorting
end inner loop
Iteration 9, inertia 824978.449380123
start iteration
done sorting
end inner loop
Iteration 10, inertia 824895.5238387219
start iteration
done sorting
en

Iteration 39, inertia 718379.2841844455
start iteration
done sorting
end inner loop
Iteration 40, inertia 718379.2841844455
center shift 0.000000e+00 within tolerance 2.414089e-05


In [12]:
print(score10, score20, score30)
# We can see the optimal k for this document collection should be less than 10.

0.07355711855910863 0.031287004955803936 -0.04060476480167929


# Verify Figure 5.13 using the algorithm in Figure 5.12 

In [14]:
def normalProb(xi, mu):
    return np.exp(-(xi-mu)**2/2)
def estep(x, clusterMu):
    probs = {}
    for i in range(len(clusterMu)):
        probs[i] = []
    for i in range(len(x)):
        prob = []
        for j in range(len(clusterMu)):
            mu = clusterMu[j]
            prob.append(normalProb(x[i], mu))
        prob = prob/sum(prob)
        for j in range(len(clusterMu)):
            probs[j].append(prob[j])
    return probs
def mstep(x, probs):
    newMean = []
    for c in probs:
        newMean.append(sum(np.array(x)*np.array(probs[c]))/sum(probs[c]))
    return newMean
x = [0, 1, 2, 4, 4]
mu = [2,3]
for step in range(4):
    print('step',step)
    if step == 0:
        print(['%.1f' % elem for elem in mu])
    else:
        print('E step')
        for c in probs:
            print(['%.1f' % elem for elem in probs[c]])
        print('M step')
        print(['%.1f' % elem for elem in mu])
    probs = estep(x, mu)
    mu = mstep(x, probs)

step 0
['2.0', '3.0']
step 1
E step
['0.9', '0.8', '0.6', '0.2', '0.2']
['0.1', '0.2', '0.4', '0.8', '0.8']
M step
['1.3', '3.3']
step 2
E step
['1.0', '0.9', '0.6', '0.0', '0.0']
['0.0', '0.1', '0.4', '1.0', '1.0']
M step
['0.9', '3.6']
step 3
E step
['1.0', '1.0', '0.7', '0.0', '0.0']
['0.0', '0.0', '0.3', '1.0', '1.0']
M step
['0.9', '3.7']


# How can the EM algorithm be used to perform a k-means clustering?

We can still use the EM algorithm with E step replaced by the hard cluster assignment rule that probability for 
one cluster is 1 if the cluster is identified in the M step, 0 otherwise. so the sum of probability equals 1 
the same as regular EM algorithm.